In [44]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 20

# Import and Set Up Spotipy

In [31]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = '78e434819ce74de2a9da9bd344483f65'
secret = '4bc5580d98ae45f087b511650c3e9d0b'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# Retrieve Data from Spotify Web API

In [ ]:
seeds = sp.recommendation_genre_seeds()
seeds['genres']

In [ ]:
import time

artist_name = []
album_name = []
album_id = []
track_name = []
track_id = []
popularity = []
release_date = []
duration_ms = []
explicit = []
external_urls = []
is_local = []
preview_url = []
track_number = []
audio_features = []
genre = []


try:
    for k,i in enumerate(range(0,1000,50)):
           
           count = 1
           for s in seeds['genres']:
                start_time = time.time()
                track_results = sp.search(q=f'year:2022 genre:{s}' , type='track', offset=i, limit=50)   
                for j, t in enumerate(track_results['tracks']['items']):
                    # Get artist data
                    artist_name.append(t['artists'][0]['name'])
                    
                    # Get album data
                    album_name.append(t['album']['name'])
                    album_id.append(t['album']['id'])
                    release_date.append(t['album']['release_date'])
                    
                    # Get track data
                    track_name.append(t['name'])
                    track_id.append(t['id'])
                    popularity.append(t['popularity'])
                    duration_ms.append(t['duration_ms'])
                    explicit.append(t['explicit'])
                    external_urls.append(t['external_urls']['spotify'])
                    is_local.append(t['is_local'])
                    preview_url.append(t['preview_url'])
                    track_number.append(t['track_number'])
                    genre.append(s)
                    
                    # Get audio features
                    af = sp.audio_features(t['id'])[0]
                    audio_features.append(af)
                    end_time = time.time()
                    
                count += 1
                print(f'iterate {k+1}/{len(range(0,1000,50))} ---- > genre : {count}/127 (time : {end_time - start_time} sec)')
                    

except:
    print(len(artist_name))

#  Load Data into DataFrame for Exploratory Data Analysis

In [38]:
import pandas as pd
track_dataframe = pd.DataFrame({
    'artist_name': artist_name,
    'album_name': album_name,
    'album_id': album_id,
    'track_name': track_name,
    'track_id': track_id,
    'popularity': popularity,
    'release_date': release_date,
    'duration_ms': duration_ms,
    'explicit': explicit,
    'external_urls': external_urls,
    'is_local': is_local,
    'preview_url': preview_url,
    'track_number': track_number,
    'audio_features': audio_features,   
    'genre' : genre
    })
print(track_dataframe.shape)


(110158, 15)


# Get audio_features

In [39]:
# Use pd.json_normalize() to convert the audio_features column to separate columns
audio_features_df = pd.json_normalize(track_dataframe['audio_features'])

# Add the new columns to the original DataFrame
track_dataframe = pd.concat([track_dataframe, audio_features_df], axis=1)

# Remove the original audio_features column
track_dataframe.drop('audio_features', axis=1, inplace=True)



In [36]:
track_dataframe.to_csv('data_5k_lignes_test.csv', index=False)

In [37]:
for i,j in enumerate( range(0,100,50)):
    print(j)


0
50


In [58]:
track_dataframe[[ 'genre', 'track_id']].duplicated().sum()

260

In [41]:
track_dataframe.to_csv('data_83k_lignes.csv')

In [45]:
track_dataframe['genre'].value_counts()

acoustic          1000
metal             1000
power-pop         1000
pop-film          1000
pop               1000
                  ... 
honky-tonk         716
grindcore          708
chicago-house      524
iranian            246
detroit-techno     236
Name: genre, Length: 113, dtype: int64